# This notebook shows how to obtain the wave function from a quantum circuit. IBM provides a simulator that could output wave function easily, the backend "statevector_simulator" in qiskit.Aer module. The example of obtaining the simulated statevector will be shown. However, in real qubits, only the weights of each computational basis can be obtained. This computation will also be shown. Please pay attention to the difference of executing on statevector_simulator and qubits in the following examples.

In [1]:
import qiskit
from qiskit import IBMQ
from qiskit import Aer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute

In [67]:
# define the backends : quantum computer simulator, statevector simulator
sim_backend=Aer.get_backend('qasm_simulator')
statevectorsim_backend=Aer.get_backend('statevector_simulator')

In [55]:
# define the backends : cloud quantum computer
accountinfo=IBMQ.stored_accounts()
qbackend=IBMQ.get_backend('ibmqx2')
print(accountinfo)

[{'token': '9293b5ea7a0029f9141900aa84ad425262f91193abfaae6a41136066fce2a3c3f9981d6933188a79dcc914a7785d620287ffeee20ebc2f01928c789e0b386977', 'url': 'https://api.quantum-computing.ibm.com/api/Hubs/ibm-q/Groups/open/Projects/main'}]


## A note for using statevector_simulator as backend:
Quote from Qiskit documentation "Note that if a circuit contains measure or reset the final statevector will be a conditional statevector after simulating wave-function collapse to the outcome of a measure or reset. " https://qiskit.org/documentation/aer/simulate_statevectors.html
The statevector_simulator can not be used simultaneously with the measurement gate. 

## The first example is a one qubit circuit.

In [80]:
# build the cirucruit
# one qubit and make superposition 0-1
q=QuantumRegister(1)
c=ClassicalRegister(1)
circ=QuantumCircuit(q,c)
circ.x(q)
circ.h(q)
# for running getting statevector from statevector_sim, there can not be measurement gate
job_statevector_sim=execute(circ, backend=statevectorsim_backend, shots=1024)
result_statevector_sim=job_statevector_sim.result()
# Other backends must need measurement gate in order to get the wave function
circ.measure(q,c)
job_sim=execute(circ,backend=Aer.get_backend('qasm_simulator'), shots=1024)
qjob=execute(circ, qbackend, shots=1024)
result_sim=job_sim.result()
qresult=qjob.result()

In [84]:
# printout the results of the three different backends
# get the wave function
print('The weights of statevector using statevector_simulator is {}'.format(result_statevector_sim.get_counts()))
print('The statevector given by statevector_simulator is {}'.format(result_statevector_sim.get_statevector(circ)))

print('The weights of statevector using qubits is {}'.format(qresult.get_counts()))
print('The weights of statevector using qasm_simulator is {}'.format(result_sim.get_counts()))

The weights of statevector using statevector_simulator is {'0': 1}
The statevector given by statevector_simulator is [ 0.70710678+0.j -0.70710678+0.j]
The weights of statevector using qubits is {'1': 483, '0': 541}
The weights of statevector using qasm_simulator is {'1': 531, '0': 493}


## The second example is a two-bit example, no entanglement.

In [104]:
# two qubit and make first 0+1, the second 0-1
q2=QuantumRegister(2)
c2=ClassicalRegister(2)
circ2=QuantumCircuit(q2,c2)
circ2.h(q2[0])
circ2.x(q2[1])
circ2.h(q2[1])
# for running getting statevector from statevector_sim, there can not be measurement gate
job_statevector_sim=execute(circ2, backend=statevectorsim_backend, shots=1024)
result_statevector_sim=job_statevector_sim.result()
circ2.measure(q2,c2)
job_sim=execute(circ2,backend=Aer.get_backend('qasm_simulator'), shots=1024)
qjob=execute(circ2, qbackend, shots=1024)
result_sim=job_sim.result()
qresult=qjob.result()

In [105]:
# printout the results of the three different backends
# get the wave function
print('The weights of statevector using statevector_simulator is {}'.format(result_statevector_sim.get_counts()))
print('The statevector given by statevector_simulator is {}'.format(result_statevector_sim.get_statevector()))

# Note that the weights do not contain the information of the sign.
print('The weights of statevector using qubits is {}'.format(qresult.get_counts()))
print('The weights of statevector using qasm_simulator is {}'.format(result_sim.get_counts()))

The weights of statevector using statevector_simulator is {'00': 1}
The statevector given by statevector_simulator is [ 0.5+0.j  0.5+0.j -0.5+0.j -0.5+0.j]
The weights of statevector using qubits is {'01': 236, '00': 266, '11': 273, '10': 249}
The weights of statevector using qasm_simulator is {'01': 273, '00': 254, '11': 242, '10': 255}
